In [42]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [43]:
# executeme

#@time begin
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

using LinearAlgebra

#end

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [97]:
Nx = 8; Ny = 8; N=Nx*Ny; q = Nx; N_cut = q
PN = 2
U = 2

2

In [98]:
basis = NLevelBasis(N)
Op = Hofstadter_SP(Nx,Ny,1/q, 0);

In [99]:
# executeme

function get_sp_op(basis, Op)
  
    H = SparseOperator(basis)

    for m in 1:N
        for n in 1:N
            H = H + Op[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

get_sp_op (generic function with 1 method)

In [100]:
H1 = get_sp_op(basis, Op)

Operator(dim=64x64)
  basis: NLevel(N=64)sparse([2, 8, 9, 57, 1, 3, 10, 58, 2, 4  …  61, 63, 7, 55, 62, 64, 8, 56, 57, 63], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  62, 62, 63, 63, 63, 63, 64, 64, 64, 64], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -0.7071067811865476 - 0.7071067811865475im, -0.7071067811865476 + 0.7071067811865475im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 1.8e-16 + 1.0im, 1.8e-16 - 1.0im, -1.0 + 0.0im, -1.0 + 0.0im, -0.7071067811865474 + 0.7071067811865477im, -0.7071067811865474 - 0.7071067811865477im, -1.0 + 0.0im, -1.0 + 0.0im], 64, 64)

In [101]:
#check op. form
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [102]:
# executeme

function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [103]:
sub_states = get_sub_states(H1, N_cut);

In [104]:
# executeme

function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [105]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [106]:
# executeme

function Subspace_Op(SP_Op, P, Pt)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [107]:
H1_sub = Subspace_Op(H1, P, Pt)

Operator(dim=8x8)
  basis: Subspace(superbasis=NLevel(N=64), states:8)
 -3.29066-2.2e-16im  -3.9e-16-3.3e-16im  …   1.0e-16-5.0e-16im
 -1.1e-16-1.1e-16im  -3.29066+2.2e-16im     -1.7e-16+2.4e-16im
 -2.5e-16-3.0e-17im  -4.0e-17+8.0e-17im      4.2e-16+2.8e-16im
  2.2e-16-2.5e-16im  -1.7e-16+2.2e-16im     -1.4e-16+8.0e-17im
  3.3e-16-8.9e-16im   1.7e-16-1.1e-16im     -2.8e-16+8.0e-17im
  2.8e-16+2.2e-16im   3.0e-17-2.2e-16im  …  -5.6e-16+0.0im
      0.0-1.1e-16im   3.0e-17-6.0e-17im     -6.0e-17-6.0e-17im
  1.7e-16+4.4e-16im  -6.7e-16-3.2e-16im     -3.29066+0.0im

In [108]:
# executeme

function get_num_sub_list(basis,P,Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(basis, m, m)
        NMP = Subspace_Op(NM, P,Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [109]:
num_sub_list = get_num_sub_list(basis,P,Pt);

In [110]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:8), states:36)

In [111]:
# executeme

function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

get_MB_Op (generic function with 1 method)

In [112]:
H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub);

In [113]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [114]:
# executeme

function get_num_mb_list(basis_mb, b_sub, num_sub_list)
    num_mb_list = []
    for m in 1:N
        NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [115]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

In [116]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U);

In [117]:
#@btime begin
@time begin
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)
end

  3.260326 seconds (21.52 M allocations: 2.087 GiB, 23.95% gc time, 14.18% compilation time)


Operator(dim=36x36)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:8), states:36)
    0.0779495+0.0im          …  -0.000255522-0.00472654im
    0.0114156-0.0181543im          0.0026259+0.00290655im
   -0.0202898-0.011048im          0.00127991-0.00780552im
   0.00999451+0.0122762im         0.00401185+0.0056385im
   -0.0035547-0.00964726im        0.00223007-0.00372999im
   -0.0015556+0.00293585im   …    0.00117939-0.0013653im
   0.00270117+0.0161275im       -0.000613702+0.00123327im
   0.00663041+0.00606462im        0.00197101-0.00381254im
   -0.0108066-0.00504781im       -0.00249395-0.000472747im
   -0.0202041+0.00672195im        0.00752816+0.0045078im
  -0.00118873+0.00123647im   …   -0.00302053+0.00388983im
  -0.00569391+0.000695725im       0.00133404+0.00487496im
 -0.000609054+0.00460875im       -0.00782134+7.42659e-5im
             ⋮               ⋱              ⋮
  -0.00249624+0.00883803im       -0.00729811-0.00299892im
   0.00230803+0.00261982im   …    -0.

In [146]:
P1 = P.data
P1t = Pt.data;

In [147]:
@time  @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n];

  0.116271 seconds (1.86 M allocations: 56.849 MiB)


In [148]:
@time b2_sub = b_sub ⊗ b_sub

  0.000013 seconds (2 allocations: 176 bytes)


[Subspace(superbasis=NLevel(N=64), states:8) ⊗ Subspace(superbasis=NLevel(N=64), states:8)]

In [149]:
@time P4re = reshape(P4, N_cut^2, N_cut^2);

  0.000025 seconds (2 allocations: 96 bytes)


In [150]:
@time Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re);

  0.000095 seconds (9 allocations: 160.938 KiB)


In [151]:
# executeme

function get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, N_cut^2, N_cut^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    #Vint_bsub2 = Operator(b2_sub,U/2*P4re)

    return Vint_bsub2
end

get_Vint_bsub2_einsum (generic function with 1 method)

In [152]:
@time begin
Vint_bsub2 = get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
end;

  0.134624 seconds (85.81 k allocations: 5.308 MiB, 98.46% compilation time)


In [153]:
@time begin
Vint_mb_einsum = manybodyoperator(basis_mb, Vint_bsub2)
end;

  1.398199 seconds (10.62 M allocations: 1.266 GiB, 17.53% gc time)


In [154]:
bcut_mb, bcut = get_Bosonic_MB_Basis(N_cut,PN)

(ManyBody(onebodybasis=NLevel(N=8), states:36), NLevel(N=8))

In [155]:
@time begin
P1 = P.data
P1t = Pt.data;

@einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

b2cut = bcut ⊗ bcut

P4re = reshape(P4, N_cut^2, N_cut^2)
Vint_bsub2 = SparseOperator(b2cut,U/2*P4re)
end;

  0.110425 seconds (1.86 M allocations: 57.006 MiB)


In [156]:
@time begin
Vint_mb_cut = manybodyoperator(bcut_mb, Vint_bsub2)
end;

  1.405574 seconds (10.62 M allocations: 1.266 GiB, 15.72% gc time)


In [132]:
function manybodyoperator_2_Hubbard(basis::ManyBodyBasis, op::SparseOpType)
    N = length(basis)
    S = length(basis.onebodybasis)
    result = SparseOperator(basis)
    occupations = basis.occupations
    rows = QuantumOpticsBase.rowvals(op.data)
    values = QuantumOpticsBase.nonzeros(op.data)
    @inbounds for column=1:S^2, j in QuantumOpticsBase.nzrange(op.data, column)
        row = rows[j]
        value = values[j]
        index = Tuple(CartesianIndices((S, S, S, S))[(column-1)*S^2 + row])
        for m=1:N, n=1:N
            # println("row:", row, " column:"column, ind_left)
            C = QuantumOpticsBase.coefficient(occupations[m], occupations[n], index[1:2], index[3:4])
            if C!=0.
                result.data[m,n] += C*value
            end
        end
    end
    return result
end

manybodyoperator_2_Hubbard (generic function with 1 method)

In [74]:
@time Vint_mb_Hubbard = manybodyoperator_2_Hubbard(basis_mb, Vint_bsub2);

  0.083780 seconds (77.49 k allocations: 4.081 MiB, 99.28% compilation time)


In [75]:
isapprox( Vint_mb_einsum, Vint_mb_Hubbard )

true

In [76]:
isapprox( Vint_mb , Vint_mb_einsum )

true

In [77]:
Vint_mb_einsum.data

6×6 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 36 stored entries:
    0.39162-1.47451e-17im  …    0.0596277-0.053036im
 -0.0623729-0.00749462im        0.0179539-0.0124782im
  0.0130024-0.0341827im         -0.031339-0.0823887im
  0.0371678+0.00269294im       -0.0849376+0.0651927im
 -0.0494625+0.0943022im       -0.00894969-0.0359646im
  0.0596277+0.053036im     …     0.351317-1.17094e-17im

In [78]:
Vint_mb_Hubbard.data

6×6 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 36 stored entries:
    0.39162-1.47451e-17im  …    0.0596277-0.053036im
 -0.0623729-0.00749462im        0.0179539-0.0124782im
  0.0130024-0.0341827im         -0.031339-0.0823887im
  0.0371678+0.00269294im       -0.0849376+0.0651927im
 -0.0494625+0.0943022im       -0.00894969-0.0359646im
  0.0596277+0.053036im     …     0.351317-1.17094e-17im

In [79]:
H_MB = H1_MB + Vint_mb_einsum
H_mb = H1_MB + Vint_mb_Hubbard

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)sparse([1, 2, 3, 4, 5, 6, 1, 2, 3, 4  …  3, 4, 5, 6, 1, 2, 3, 4, 5, 6], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  5, 5, 5, 5, 6, 6, 6, 6, 6, 6], ComplexF64[-5.072481889080085 + 7.0e-17im, -0.06237291741456313 - 0.00749462289574094im, 0.01300238399161972 - 0.03418266124713084im, 0.03716777427851 + 0.00269293504848635im, -0.04946251729897929 + 0.09430216055688354im, 0.05962766535723585 + 0.0530359855118317im, -0.06237291741456313 + 0.00749462289574085im, -5.2544047986689835 + 4.0e-17im, 0.01265677755383486 - 0.05086158930894007im, -2.7e-16 - 1.1e-16im  …  0.08820862573245139 - 0.01059899734402867im, -9.0e-17 - 1.1e-16im, -5.173798431606525 + 1.2e-16im, -0.00894969323628626 + 0.03596457470227668im, 0.05962766535723585 - 0.0530359855118317im, 0.01795385454455051 - 0.01247819123292165im, -0.0313389604933155 - 0.08238866588393709im, -0.08493763881226458 + 0.06519270465165757im, -0.00894969323628638 

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [80]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [81]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-5.297434948471091, -5.297434948471087, -5.297434948471086, -4.964101615137758, -4.9641016151377535, -4.964101615137753]
[-5.297434948471091, -5.297434948471087, -5.297434948471086, -4.964101615137758, -4.9641016151377535, -4.964101615137753]
[-5.340971176014522, -5.340971176014518, -5.34097117601451, -5.066168265618836, -5.066168265618833, -5.06616826561883, -1.8128503544788697, -1.8128503544788683, -1.8128503544788683, -1.704570336931171]